In [0]:
from google.colab import drive

In [0]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cd drive


In [0]:
!ls

drive  sample_data


In [0]:
cd drive


/content/drive


In [0]:
!ls

'My Drive'


In [0]:
cd 'My Drive'

/content/drive/My Drive


In [0]:
!ls


'20180405 (1).gpx.gdoc'
 20180405.gpx.gdoc
 6_Nonregular_Languages.gdoc
 6_Nonregular_Languages.pdf
'ASE2 M2018 Final Marks.gsheet'
 Blank_Project.gscript
 CGM
'CGM Project'
 Charades_v1_rgb.tar
 Classroom
'Colab Notebooks'
 DE_mid2.gdoc
 DE_mid2.pdf
 eis.odt.gdoc
 first_evaluation
 Function
'GE-assignment01 (1).gdoc'
'GE-assignment01 (2).gdoc'
 GE-assignment01.docx
 GE-assignment01.gdoc
 GE-assignment01.pdf
'Getting started.pdf'
'GPSLogger for Android'
 IMG_20170526_231018_988.jpg
'IMG_20170526_231018_988 - JUNAID NZ.jpg'
'junaid (1).odt.gdoc'
'junaid (2).odt.gdoc'
 Junaid_ITWS3.zip
 Junaidnzphoto.jpg
'JUNAID_NZ_RESUME (1).pdf'
'JUNAID_NZ_RESUME - JUNAID NZ (1).pdf'
'JUNAID_NZ_RESUME - JUNAID NZ.pdf'
 JunaidNZ_Resume.pdf
 JUNAID_NZ_RESUME.pdf
 Junaidnzsignature.jpg
 junaid.odt.gdoc
'Junaid_Resume_2020 (1).pdf'
'Junaid_Resume_2020 (2).pdf'
'Junaid_Resume_2020 (3).pdf'
'Junaid_Resume_2020 (4).pdf'
'Junaid_Resume_2020 - JUNAID NZ (1).pdf'
'Junaid_Resume_2020 - JUNAID NZ.pdf'
 Junaid_Resu

In [0]:
cd 'CGM Project'

/content/drive/My Drive/CGM Project


In [0]:
!ls

 charades-algorithms-master.zip   Charades_v1_train.gsheet	 my2.png
 charades_v1_rgb		 'Copy of image_embedding.npy'	 my.png
 Charades_v1_rgb.tar		  dataset			 vid_ids.csv
'Charades_v1_train (1).gsheet'	  frameextractor.ipynb
 Charades_v1_train.csv		  img_emd.npy


In [0]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math

In [0]:
df = pd.read_csv('Charades_v1_train.csv')

In [0]:
df2 = pd.read_csv("vid_ids.csv")
vids = set()
for i in df2["0"]:
  vids.add(i)
script_dict={}
for rows in df.itertuples(): 
  if rows.id in vids:
    script_dict[rows.id]=rows.script
print(script_dict)


{'46GP8': 'A person cooking on a stove while watching something out a window.', 'N11GT': 'One person opens up a folded blanket, then sneezes and leaves the stairs with a broom.', 'KRF68': 'A person runs into their laundry room. They grasp the doorknob and close the door. The person starts dressing then leaves holding their phone.', 'MJO7C': 'A person runs into their pantry holding a bottle of medicine. They grab the doorknob and close the door while watching a video on their phone.', 'S6MPZ': 'A person is eating at the desk and lying the phone down.', '7HVU8': 'A person is throwing items into a bag then starts fixing their clothes.', 'MCQO5': 'A person is undressing in the closet.  They put their clothes back into the wardrobe and drink a glass of water while looking at a picture.', 'VPIYF': 'A person is standing by a shelf and grasping his phone off of it.', 'JSUF4': 'A person is sweeping up with a broom and putting things back in their place. They take break on the sofa and start dri

In [0]:
path = '/charades_v1_rgb/'
ar = []

In [0]:
dir_path = os.getcwd() + path
ar = next(os.walk(dir_path))[1]

In [0]:
!ls

 charades-algorithms-master.zip   frameextractor.ipynb
 charades_v1_rgb		  image_embedding_bak.npy
 Charades_v1_rgb.tar		  image_embedding.npy
'Charades_v1_train (1).gsheet'	  img_emd.npy
 Charades_v1_train.csv		  my2.png
 Charades_v1_train.gsheet	  my.png
'Copy of image_embedding.npy'	  vid_ids.csv
 dataset


In [0]:
def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

In [0]:
action_frames = {}
label_count = {}

for j in range(len(ar)):
  action_frames[ar[j]] = {}

for i in range(len(df)):
  for j in range(len(ar)):
    if(df.loc[i, "id"] == ar[j]):
      actions = df.loc[i, "actions"].split(";")
      label_count[ar[j]] = len(actions)
      v_id = df.loc[i, "id"]
      frames = []
      video_path = os.getcwd() + path + v_id
      print(video_path)
      for r, d, f in os.walk(video_path):     
        for files in f:
          nar = load_image(video_path + '/' + files)
          frames.append(nar)
      frames = frames[:-1]
      print(len(frames))
      req_frames = {}
      for k in range(len(actions)):
        label, start, end = actions[k].split(' ')
        req_frames[label] = []
      for p in range(len(frames)):
        for k in range(len(actions)):
          label, start, end = actions[k].split(' ')
          startframe = math.floor(float(start) * 24)
          endframe = math.floor(float(end) * 24)
          if(p >= startframe and p <= endframe):
            req_frames[label].append(frames[p])
      print("DONE")
      imp_frames = {}
      for k in range(len(actions)):
        label, start, end = actions[k].split(' ')
        imp_frames[label] = []
        for p in range(len(req_frames[label])):
          matches = 0
          print(p)
          if(p % 3 == 0):
            imp_frames[label].append(req_frames[label][p])
          print(len(req_frames[label]), len(imp_frames[label]))
      action_frames[ar[j]] = imp_frames
      for actions in action_frames[ar[j]]:
        print((np.array(action_frames[ar[j]][actions]).shape))
print(action_frames)
print(label_count)
              
  

/content/drive/My Drive/CGM Project/charades_v1_rgb/4SN6Z


KeyboardInterrupt: ignored

In [0]:
print(df.loc[0, "id"])

46GP8
